In [1]:
from importlib.resources import path
import re
from bs4 import BeautifulSoup

import shutil
import os
from rdflib import Graph
from shutil import copy
import xml.etree.ElementTree as ET
from matplotlib import lines
import glob
import pandas as pd
import pyarabic.araby

In [2]:
files = glob.glob('D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\**\\*.xml', recursive = True)
files

['D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S000A MuqadimaA\\sure_0_section.xml',
 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S001M\\sure_1_section_1.1-1.14.xml',
 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S002A\\sure_2_section_2.1-2.42.xml',
 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S002A\\sure_2_section_2.113-2.165.xml',
 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S002A\\sure_2_section_2.166-2.210.xml',
 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S002A\\sure_2_section_2.211-2.263.xml',
 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S002A\\sure_2_section_2.264-2.297.xml',
 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S002A\\sure_2_section_2.298-2.344.xml',
 'D:\\University\\semest

In [43]:
def sections(inputfile):
    import xml.etree.ElementTree as ET
    with open(inputfile, 'r',encoding='utf8') as f:
        xml_file_as_string = f.read()
    ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
    xml_root = ET.fromstring(xml_file_as_string)
    ns = xml_root.tag
    ns = ns[:ns.rfind('}') + 1]
    ana_tags_statistics = dict()
    # text/body/div/p/ana
    sections_list = []
    try:
        for paragraph_node in xml_root.find(ns + "text").find(ns + "body").iter(ns + "div"):
            # for div_node in paragraph_node.findall(ns+"p"):
            paragraph_type_attrib = paragraph_node.get("type")
            paragraph_n_attrib = paragraph_node.get("n")

            if paragraph_type_attrib == "section":
                sections_list.append(paragraph_n_attrib)

            if paragraph_type_attrib == "chapter" and "section" not in xml_file_as_string:
                sections_list.append(paragraph_n_attrib)
            # print(type(paragraph_n_attrib))
    except:
        try:
            try:
                for paragraph_node in xml_root.find(ns + "text").find(ns + "body").find(ns + "div").iter(ns + "div"):
                    # for div_node in paragraph_node.findall(ns+"p"):
                    paragraph_type_attrib = paragraph_node.get("type")
                    paragraph_n_attrib = paragraph_node.get("n")

                    if paragraph_type_attrib == "section":
                        sections_list.append(paragraph_n_attrib)
                    # print(type(paragraph_n_attrib))
            except:
                for paragraph_node in xml_root.find(ns + "text").find(ns + "body").iter(ns + "div"):
                    # for div_node in paragraph_node.findall(ns+"p"):
                    paragraph_type_attrib = paragraph_node.get("type")
                    paragraph_n_attrib = paragraph_node.get("n")

                    if paragraph_type_attrib == "chapter":
                        print("found")
                        sections_list.append(paragraph_n_attrib)
        except:
            print("Error1")

    # print(sections_list[0],sections_list[-1])
    # sure = sections_list[0]
    # print(sure[0])
    return sections_list

In [44]:
def  extract_paragraphs_2(input_file, list_of_list_for_cvs):
    #header = parse_paragraph_quran_n(input_file)
    sura_number = sections(input_file)
    with open(input_file, 'r',encoding='utf8') as f: 
        data = f.read() 
        Bs_data = BeautifulSoup(data, "xml") 

        # find all sub_sections
        sub_sections = Bs_data.find_all('div')
        sub_sections_liste = []
        for i in sub_sections:
            try:
                an = (i['n'])
                sub_sections_liste.append(an)
            except: 
                #print(i)
                sub_sections_liste.append("no ayat number")
                continue
       # print(sub_sections_liste)
        #sub_sections = list(sub_sections)
        #print(sub_sections)

        
        list_of_list_paragraphs = []
        list_of_list_topics = []
        list_of_list_subtopics = []
        list_of_list_xml_id = []
        list_of_list_HadithNotHadith = []
        list_of_list_ayat_numbers = []
        list_of_list_rawis = []
        list_of_list_sahabis = []
        list_of_list_shaykhs = []
        list_of_list_persons = []
        list_of_list_locations = []
        list_of_list_organizations = []
        list_of_list_times = []
        list_of_list_others = []
        list_of_list_heads_30 = []
        

        # zahler 
        zaehler_topics = 0
        zaehler_subtopics = 0


        previous_ayat = ""



        # delte double sub_sections
        '''
        list_of_sub_sections_reverse = sub_sections[::-1]
        list_of_subsections_clear = [] 
        liste_new = []
        end_flag = 0
        for element in list_of_sub_sections_reverse:
            element_str = str(element)
            check_line = element_str
            for element_new in list_of_subsections_clear:
                if check_line == str(element_new):
                    end_flag = 1
                    break
            if end_flag == 1:
                end_flag = 0
                continue
            else:
                list_of_subsections_clear.append(element)
        list_of_subsections_clear_reverse = list_of_subsections_clear[::-1]

        #print(list_of_subsections_clear_reverse)
        
        #print(sub_sections)
        #print(type(sub_sections))
        '''

        for i in sub_sections: #sub_sections
            #print("Hallo")
            #print(i)

            # head = 30
            heads_30 = [] 
            a_numb = "No Ayat Number"
            
            heads_first = i.find_all('head', {'type' : '30'})
            for head in heads_first:
                ayat_number = head.find('quote')
                try:
                    a_numb = (ayat_number['n'])
                    previous_ayat = a_numb
                except:
                    a_numb = "No Ayat Number"
                    continue
              
            if a_numb == "No Ayat Number":
                a_numb =  a_numb + " (" + previous_ayat + ")"

        
            # paragraphs
            paragraphs = [] 
            paragraphs_first = i.find_all('p')
            for i in paragraphs_first:
                if i == "\n":
                    paragraphs.append("No Text") 
                else:
                    paragraphs.append(i) 
            list_of_list_paragraphs.append(paragraphs)


            # ayat numbers add to paragraph
            for i in range(len(paragraphs)):
                heads_30.append(a_numb)
            list_of_list_heads_30.append(heads_30)



            # topics
            liste_topics = []
            for i in paragraphs:
                try:
                    a = (i['ana'])
                    zaehler_topics = zaehler_topics + 1
                    liste_topics.append(a)
                except: 
                    #print(i)
                    liste_topics.append("no topic")
                    continue
            list_of_list_topics.append(liste_topics)

            # subtopics
            subtopics = []
            subtopics_of_paragraph = []
            for i in paragraphs:
                paragraph_subtopics = i.find_all('seg')
                for a in paragraph_subtopics:
                    try:
                        s = (a['ana'])
                        zaehler_subtopics = zaehler_subtopics + 1
                        subtopics_of_paragraph.append(s)
                    except: 
                        subtopics_of_paragraph.append("no subtopic")
                    continue
                subtopics.append(subtopics_of_paragraph)
                subtopics_of_paragraph = []
            list_of_list_subtopics.append(subtopics)
            #print(list_of_list_subtopics)

            # ayat numbers in paragraph
            ayat_numbers = []
            ayat_numbers_of_paragraph = []
            for i in paragraphs:
                paragraph_ayat_numbers = i.find_all('quote', {'type': 'quran'})
                for a in paragraph_ayat_numbers:
                    try:
                        s = (a['n'])
                        ayat_numbers_of_paragraph.append(s)
                    except: 
                        ayat_numbers_of_paragraph.append("no ayat number")
                    continue
                ayat_numbers.append(ayat_numbers_of_paragraph)
                ayat_numbers_of_paragraph = []
            list_of_list_ayat_numbers.append(ayat_numbers)


            # shaykh
            shaykhs_of_paragraph = []
            shaykhs = []
            for i in paragraphs:
                paragraph_shaykhs = i.find_all('persName', {'ana' : 'shaykh'} )
                shaykhs_of_paragraph = []
                for a in paragraph_shaykhs:
                    if paragraph_shaykhs == []:
                        shaykhs_of_paragraph.append("no shaykh")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        shaykhs_of_paragraph.append(a)
                shaykhs.append(shaykhs_of_paragraph)
            list_of_list_shaykhs.append(shaykhs)


            # rawi 
            rawis_of_paragraph = []
            rawis = []
            for i in paragraphs:
                paragraph_rawis = i.find_all('persName', {'ana' : 'rawi'} )
                rawis_of_paragraph = []
                for a in paragraph_rawis:
                    if paragraph_rawis == []:
                        rawis_of_paragraph.append("no rawi")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        rawis_of_paragraph.append(a)
                rawis.append(rawis_of_paragraph)
            list_of_list_rawis.append(rawis)
        


            # sahabi
            sahabis_of_paragraph = []
            sahabis = []
            for i in paragraphs:
                paragraph_sahabis = i.find_all('persName', {'ana' : 'sahabi'} )
                sahabis_of_paragraph = []
                for a in paragraph_sahabis:
                    if paragraph_sahabis == []:
                        sahabis_of_paragraph.append("no sahabi")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        sahabis_of_paragraph.append(a)
                sahabis.append(sahabis_of_paragraph)
            list_of_list_sahabis.append(sahabis)


            # NER person
            persons_of_paragraph = []
            persons = []
            for i in paragraphs:
                paragraph_persons = i.find_all('name', {'role' : 'person'} )
                #print(paragraph_persons)
                persons_of_paragraph = []
                for a in paragraph_persons:
                    if paragraph_persons == []:
                        persons_of_paragraph.append("no person")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        persons_of_paragraph.append(a)
                #zaehler_person = zaehler_person + len(persons_of_paragraph)
                persons.append(persons_of_paragraph)
            list_of_list_persons.append(persons)
            #print(list_of_list_persons)
            


            # NER location
            locations_of_paragraph = []
            locations = []
            for i in paragraphs:
                paragraph_locations = i.find_all('name', {'role' : 'location'} )
                locations_of_paragraph = []
                for a in paragraph_locations:
                    if paragraph_locations == []:
                        locations_of_paragraph.append("no location")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        locations_of_paragraph.append(a)
                locations.append(locations_of_paragraph)
            list_of_list_locations.append(locations)
            

            # NER organization
            organizations_of_paragraph = []
            organizations = []
            for i in paragraphs:
                paragraph_organizations = i.find_all('name', {'role' : 'organization'} )
                organizations_of_paragraph = []
                for a in paragraph_organizations:
                    if paragraph_organizations == []:
                        organizations_of_paragraph.append("no organization")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        organizations_of_paragraph.append(a)
                organizations.append(organizations_of_paragraph)
            list_of_list_organizations.append(organizations)
            


            # NER time
            times_of_paragraph = []
            times = []
            for i in paragraphs:
                paragraph_times = i.find_all('name', {'role' : 'time'} )
                times_of_paragraph = []
                for a in paragraph_times:
                    if paragraph_times == []:
                        times_of_paragraph.append("no times")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        times_of_paragraph.append(a)
                times.append(times_of_paragraph)
            list_of_list_times.append(times)
            

            # NER other
            others_of_paragraph = []
            others = []
            for i in paragraphs:
                paragraph_others = i.find_all('name', {'role' : 'other'} )
                others_of_paragraph = []
                for a in paragraph_others:
                    if paragraph_others == []:
                        others_of_paragraph.append("no others")
                    else:
                        a = str(a.text)
                        a = a.replace('\t', '')
                        a = a.replace('\n', ' ')
                        a = re.sub('\s+',' ',a)
                        a = a.strip()
                        others_of_paragraph.append(a)
                others.append(others_of_paragraph)
            list_of_list_others.append(others)
            #print(list_of_list_others)


            # xml:id
            xml_id = []
            for i in paragraphs:
                try:
                    b = i['xml:id']
                    xml_id.append(b)
                except: 
                    #print(i)
                    xml_id.append("no paragraph id")
                    continue
            list_of_list_xml_id.append(xml_id)
            #print(list_of_list_xml_id)

            # hadith / nothadith
            hadith_list = []
            for i in paragraphs:
                try:
                    c = i['n']
                    hadith_list.append(c)
                except:
                    hadith_list.append("Hadith Not Defined")
            list_of_list_HadithNotHadith.append(hadith_list)


            
                

        #print(zaehler_person)
        # sura number
        #sura_number = Bs_data.find_all('div', {'type':'chapter' })
        suraNumber = sura_number[0]
        sura_number_list = suraNumber.split(".")
        suraNumber = sura_number_list[0]
        #print(suraNumber)


       # print(len(sub_sections))
        #print(len(list_of_list_paragraphs))
        #print(len(list_of_list_topics))
        #print(len(list_of_list_xml_id))
        #print(len(list_of_list_rawis))


        with open ("file2", "a+",encoding='utf8') as target:
            
            
            for a in range(len(sub_sections_liste)):
                paragraphs_li = list_of_list_paragraphs[a]
                liste_topics_li = list_of_list_topics[a]
                subtopics_li = list_of_list_subtopics[a]
                xml_id_li = list_of_list_xml_id[a]
                list_ofHadith_li = list_of_list_HadithNotHadith[a]
                ayat_numbers_li = list_of_list_ayat_numbers[a]
                rawis_li = list_of_list_rawis[a]
                sahabis_li = list_of_list_sahabis[a]
                shaykhs_li = list_of_list_shaykhs[a]
                persons_li = list_of_list_persons[a]
                locations_li = list_of_list_locations[a]
                organisations_li = list_of_list_organizations[a]
                times_li = list_of_list_times[a]
                others_li = list_of_list_others[a]
                ayat_heads_li = list_of_list_heads_30[a]

                #row_cvs = []

                for i in range(len(list_of_list_paragraphs[a])):
                    row_cvs = []

                    # output xml:id
                    xmliD = str(xml_id_li[i])
                    #print(xmliD)
                    target.write(xmliD)
                    target.write("\n")
                    row_cvs.append(xmliD)

                    # output paragraphes
                    paragraph = str(paragraphs_li[i].text)
                    if paragraph == "":
                        paragraph = "No Text"
                    paragraph = paragraph.replace('\t', '')
                    paragraph = paragraph.replace('\n', ' ')
                    paragraph = re.sub('\s+',' ',paragraph)
                    paragraph = paragraph.strip()
                    target.write(paragraph)
                    row_cvs.append(paragraph)
                    target.write("\n")

                    # (sub-)section:
                    target.write(sub_sections_liste[a])
                    row_cvs.append(sub_sections_liste[a])
                    target.write("\n")

                    # sura number:
                    target.write(suraNumber)
                    target.write("\n")
                    row_cvs.append(suraNumber)

                    # ayat number
                    ayat_n = ayat_heads_li[i]
                    
                    print(ayat_n)
                    target.write(ayat_n)
                    row_cvs.append(ayat_n)
                    
                    target.write("\n")



                    # ayat numbers in paragraphs 
                    ayat = ayat_numbers_li[i]
                    aya_row = []
                    if ayat == []:
                        aya = "no aya"
                        target.write(aya)
                        row_cvs.append(aya)
                    else:
                        ayat = set(ayat)
                        for aya in ayat:
                            aya = aya.replace("yes", "")
                            aya = aya.replace(",", " ")
                            if aya == "":
                                aya = "no aya"
                            target.write(aya)
                            aya_row.append(aya+" ")
                            target.write(" ")
                        aya_row = set(aya_row)
                        aya_row = list(aya_row)
                        aya_row.sort()
                        aya_row_string = " ".join(aya_row)
                        row_cvs.append(aya_row_string)
                    target.write("\n")
                    target.write("\n")




                    # output topics
                    topics = str(liste_topics_li[i])
                    topics = topics.replace("yes", "")
                    if topics == "":
                            topics = "no topic"
                    target.write(topics)
                    row_cvs.append(topics)
                    target.write("\n")

                    # output subtopics
                    subt = subtopics_li[i]
                    subtopic_row = []
                    if subt == []:
                        subtopic = "no subtopic"
                        target.write(subtopic)
                        row_cvs.append(subtopic)
                    else:
                        subt = set(subt)
                        for subtopic in subt:
                            subtopic = subtopic.replace("yes", "")
                            subtopic = subtopic.replace(",", " ")
                            if subtopic == "":
                                subtopic = "no subtopic"
                            target.write(subtopic)
                            subtopic_row.append(subtopic+","+" ")
                            target.write(" ")
                        subtopic_row = set(subtopic_row)
                        subtopic_row = list(subtopic_row)
                        subtopic_row.sort()
                        subtopic_row_string = " ".join(subtopic_row)
                        row_cvs.append(subtopic_row_string)
                    target.write("\n")
                    target.write("\n")


                    # output sahabi
                    sah = sahabis_li[i]
                    #print(sah)
                    sahabi_row = []
                    if sah == []:
                        sahabi = "no sahabi"
                        target.write(sahabi)
                        row_cvs.append(sahabi)
                    else:
                        #sah = set(sah)
                        for sahabi in sah:
                            #print(sahabi)
                            #rawi = rawi.replace("yes", "")
                            #rawi = rawi.replace(",", " ")
                            if sahabi == "":
                                sahabi = "no sahabi"
                            target.write(sahabi)
                            sahabi_row.append(sahabi+ ","+" ")
                            target.write(" ")
                        #sahabi_row = set(sahabi_row)
                        #sahabi_row = list(sahabi_row)
                        #sahabi_row.sort()
                        sahabi_row_string = " ".join(sahabi_row)
                        row_cvs.append(sahabi_row_string)
                        #print(sahabi_row_string)
                    target.write("\n")
                    target.write("\n")

                    # output rawis
                    raw = rawis_li[i]
                    rawi_row = []
                    if raw == []:
                        rawi = "no rawi"
                        target.write(rawi)
                        row_cvs.append(rawi)
                    else:
                        #raw = set(raw)
                        for rawi in raw:
                            #rawi = rawi.replace("yes", "")
                            #rawi = rawi.replace(",", " ")
                            if rawi == "":
                                rawi = "no rawi"
                            target.write(rawi)
                            rawi_row.append(rawi+ ","+" ")
                            target.write(" ")
                        #rawi_row = set(rawi_row)
                        #rawi_row = list(rawi_row)
                        #rawi_row.sort()
                        rawi_row_string = " ".join(rawi_row)
                        row_cvs.append(rawi_row_string)
                    target.write("\n")
                    target.write("\n")




                    # output shaykhs
                    sahay = shaykhs_li[i]
                    #print(sah)
                    shaykh_row = []
                    if sahay == []:
                        shaykh = "no shaykh"
                        target.write(shaykh)
                        row_cvs.append(shaykh)
                    else:
                        for shaykh in sahay:
                            if shaykh == "":
                                shaykh = "no shaykh"
                            target.write(shaykh)
                            shaykh_row.append(shaykh+ ","+" ")
                            target.write(" ")
                        shaykh_row_string = " ".join(shaykh_row)
                        row_cvs.append(shaykh_row_string)
                    target.write("\n")
                    target.write("\n")


                    # output Ner persons
                    per = persons_li[i]
                    person_row = []
                    if per == []:
                        person = "no person"
                        target.write(person)
                        row_cvs.append(person)
                    else:
                        for person in per:
                            if person == "":
                                person = "no person"
                            target.write(person)
                            person_row.append(person+ ","+" ")
                            target.write(" ")
                        person_row = set(person_row)
                        person_row = list(person_row)
                        person_row.sort()
                        person_row_string = " ".join(person_row)
                        row_cvs.append(person_row_string)
                    target.write("\n")
                    target.write("\n")


                    # output Ner locations
                    loc = locations_li[i]
                    location_row = []
                    if loc == []:
                        location = "no location"
                        target.write(location)
                        row_cvs.append(location)
                    else:
                        for location in loc:
                            if location == "":
                                location = "no location"
                            target.write(location)
                            location_row.append(location+ ","+" ")
                            target.write(" ")
                        location_row = set(location_row)
                        location_row = list(location_row)
                        location_row.sort()
                        location_row_string = " ".join(location_row)
                        row_cvs.append(location_row_string)
                    target.write("\n")
                    target.write("\n")


                    # output Ner organisations
                    org = organisations_li[i]
                    organisation_row = []
                    if org == []:
                        organisation = "no organisation"
                        target.write(organisation)
                        row_cvs.append(organisation)
                    else:
                        for organisation in org:
                            if organisation == "":
                                organisation = "no organisation"
                            target.write(organisation)
                            organisation_row.append(organisation+ ","+" ")
                            target.write(" ")
                        organisation_row = set(organisation_row)
                        organisation_row = list(organisation_row)
                        organisation_row.sort()
                        organisation_row_string = " ".join(organisation_row)
                        row_cvs.append(organisation_row_string)
                    target.write("\n")
                    target.write("\n")


                    # output Ner time
                    tim = times_li[i]
                    time_row = []
                    if tim == []:
                        time = "no time"
                        target.write(time)
                        row_cvs.append(time)
                    else:
                        for time in tim:
                            if time == "":
                                time = "no time"
                            target.write(time)
                            time_row.append(time+ ","+" ")
                            target.write(" ")
                        time_row = set(time_row)
                        time_row = list(time_row)
                        time_row.sort()
                        time_row_string = " ".join(time_row)
                        row_cvs.append(time_row_string)
                    target.write("\n")
                    target.write("\n")


                    # output Ner other
                    oth = others_li[i]
                    other_row = []
                    if oth == []:
                        other = "no others"
                        target.write(other)
                        row_cvs.append(other)
                    else:
                        for other in oth:
                            if other == "":
                                other = "no others"
                            target.write(other)
                            other_row.append(other+ ","+" ")
                            target.write(" ")
                        other_row = set(other_row)
                        other_row = list(other_row)
                        other_row.sort()
                        other_row_string = " ".join(other_row)
                        row_cvs.append(other_row_string)
                    target.write("\n")
                    target.write("\n")

                    # hadith / not Hadith
                    hadith = str(list_ofHadith_li[i])
                    if hadith == "":
                        hadith = "No Hadith definition"
                    hadith = hadith.replace('\t', '')
                    hadith = hadith.replace('\n', ' ')
                    hadith = re.sub('\s+',' ',hadith)
                    hadith = hadith.strip()
                    target.write(hadith)
                    row_cvs.append(hadith)
                    target.write("\n")

                    list_of_list_for_cvs.append(row_cvs)

    # clear result/ delete double paragraphs
    list_of_list_for_cvs_reverse = list_of_list_for_cvs[::-1]
    list_of_list_for_cvs_clear = []
    end_flag = 0
    for liste in list_of_list_for_cvs_reverse:
        if list_of_list_for_cvs_clear == []:
            list_of_list_for_cvs_clear.append(liste)
        else:
            check_line = liste[1]
            for liste_new in list_of_list_for_cvs_clear:
                if check_line == liste_new[1]:
                    end_flag = 1
                    break
            if end_flag == 1:
                end_flag = 0
                continue
            else:
                list_of_list_for_cvs_clear.append(liste)
    list_of_list_for_cvs_clear_reverse = list_of_list_for_cvs_clear[::-1]

    return list_of_list_for_cvs_clear_reverse, zaehler_topics, zaehler_subtopics
    

In [3]:
files.index('D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S100M\\sure_100_section_100.1-100.1.xml')
files[196]

'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S100M\\sure_100_section_100.1-100.1.xml'

In [4]:
with open(files[196], 'r', encoding='utf8') as f: 
    xml_file_as_string = f.read()
Bs_data = BeautifulSoup(xml_file_as_string, "xml")

In [5]:
# list_i = []
# import re

# tag = Bs_data.find_all("name")
# root = ET.fromstring(xml_file_as_string)
# for i in tag:
# #     print(i)
#     O = i.get("role")
#     if O in "organiztion":
# #         print(i)
#         list_i.append(i)
# #         xml_data = str(i)

# s = set(list_i)
# len(s)

In [6]:
# s

In [7]:
list_i = []
import re

tag = Bs_data.find_all("name")
root = ET.fromstring(xml_file_as_string)
for i in tag:
#     print(i)
#     O = i.get("role")
#     if O == "organization":
#         print(i)
    list_i.append(i)
#         xml_data = str(i)

s = set(list_i)
len(s)

41

In [8]:
s

{<name role="location">الكوفة</name>,
 <name role="location">الْحِجْرِ</name>,
 <name role="location">الْمُزْدَلِفَةِ</name>,
 <name role="location">بَدْرٌ</name>,
 <name role="location">سِقَايَةِ
 								زَمْزَمَ</name>,
 <name role="location">عَرَفَةَ</name>,
 <name role="location">مُزْدَلِفَةَ</name>,
 <name role="location">مِنًى</name>,
 <name role="organization"><seg ana="kalamterm yes">الْكُفَّارِ</seg></name>,
 <name role="organization">أهل التأويل</name>,
 <name role="organization">أهل العربية</name>,
 <name role="organization">أَهْلُ
 							التَّأْوِيلِ</name>,
 <name role="organization">أَهْلُ التَّأْوِيلِ</name>,
 <name role="organization">البصريين</name>,
 <name role="organization">عَرَبِ</name>,
 <name role="organization">نحويي</name>,
 <name role="other">سورة ( إبراهيم</name>,
 <name role="other">سورة والعاديات</name>,
 <name role="person"><seg ana="qari">عَبْدِ اللَّهِ</seg></name>,
 <name role="person">ابن عباس</name>,
 <name role="person">ابْنَ عَبَّاسٍ</name>,
 <nam

In [9]:
list_i = []
import re
a = ['']



for xml_f in a:
    # extracting info from xml
    with open(xml_f, 'r', encoding='utf8') as f:     
        xml_file_as_string = f.read()

    Bs_data = BeautifulSoup(xml_file_as_string, "xml")
    tag = Bs_data.find_all("head")
    root = ET.fromstring(xml_file_as_string)
    for i in tag:
    #     print(i)
        O = i.get("quote")
        if O == "quran":
            print(processText(removeDiacritics(str(i))))
            print('\n\n')
            list_i.append(processText(removeDiacritics(i.text)))

s = set(list_i)
s = list(s)
s

[]

In [10]:

# print(list_i[0])

In [11]:
import re

def remove_numbers_and_slashes(text):
    # Use regular expression to remove numbers and slashes
    cleaned_text = re.sub(r'[0-9/-]', '', text)
    return cleaned_text

def remove_leading_spaces_list(input_list):
    return [item.strip() for item in input_list]

# text processing function
def removeDiacritics(text):
    for a in pyarabic.araby.DIACRITICS:
        text = text.replace(a, '')
    return text


def processText(text):
    text = str(text)
    text = text.replace('\t', '')
    text = text.replace('\n', ' ')

    while text.find('  ') != -1:
        text = text.replace('  ', ' ')

    text = text.strip()

    return text


def removeSpace(text: str) -> str:
    while text.find(' ') != -1:
        text = text.replace(' ', '')

    return text

from collections import Counter

def lists_have_same_elements(list1, list2):
    counter1 = Counter(list1)
    counter2 = Counter(list2)
    
    return counter1 == counter2

In [13]:
# querying from graph
    
# Create a new RDF graph
graph = Graph()

graph_file = 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\A-Box\\Tafsir Al-Tabari A-Box S55.owl'


# Parse the RDF data from a file or URL
graph.parse(graph_file)

# Define the SPARQL query
query = """
PREFIX : <http://www.tafsirtabari.com/ontology#>
PREFIX W3:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT *
WHERE {

 ?k rdf:type :Chapter.
?k :containsVerse ?v.
?v :hasText ?t.
}
"""
R = []
# Execute the SPARQL query
results = graph.query(query)
# Process the query results
for row in results:
    # Access the query variables
    s = row["k"]
    n = row["v"]
    t = row["t"]
    
    print(f"s: {s}, n: {n}, t: {t}")
    R.append(str(t))

hadiths_from_graph = list(set(R))

s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.com/ontology#V055:003, t: خَلَقَ الْإِنْسَانَ
s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.com/ontology#V055:004, t: عَلَّمَهُ الْبَيَانَ
s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.com/ontology#V055:005, t: الشَّمْسُ وَالْقَمَرُ بِحُسْبَانٍ
s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.com/ontology#V055:001, t: بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ الرَّحْمَٰنُ
s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.com/ontology#V055:002, t: عَلَّمَ الْقُرْآنَ
s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.com/ontology#V055:006, t: وَالنَّجْمُ وَالشَّجَرُ يَسْجُدَانِ
s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.com/ontology#V055:007, t: وَالسَّمَاءَ رَفَعَهَا وَوَضَعَ الْمِيزَانَ
s: http://www.tafsirtabari.com/ontology#C_055, n: http://www.tafsirtabari.

In [14]:
# text cleaning of xml and graph

#graph
cleaned_hadiths_from_graph = []
for i in hadiths_from_graph:
    a = remove_numbers_and_slashes(i)
    cleaned_hadiths_from_graph.append(removeDiacritics(a))

# xml
list_i_h = []
for i in list_i:
    filtered_list = remove_numbers_and_slashes(i)
    list_i_h.append(filtered_list)

# removing leading spaces
Hadiths_from_xml = remove_leading_spaces_list(list_i_h)

cleaned_hadiths_from_graph.sort()
Hadiths_from_xml.sort()



In [17]:
# manual comparing
print(cleaned_hadiths_from_graph[0])
print(Hadiths_from_xml[1])
print(cleaned_hadiths_from_graph[0] == Hadiths_from_xml[0])

ألا تطغوا في الميزان


IndexError: list index out of range

In [78]:
print(len(cleaned_hadiths_from_graph),len(Hadiths_from_xml))
lists_have_same_elements(cleaned_hadiths_from_graph,Hadiths_from_xml)

77 26


False

In [79]:
print(cleaned_hadiths_from_graph[8])
print('----------------')
print(Hadiths_from_xml[7])

إن كاد ليضلنا عن آلهتنا لولا أن صبرنا عليها  وسوف يعلمون حين يرون العذاب من أضل سبيلا
----------------
حنت إلي النخلة القصوى فقلت لها حجر حرام ألا تلك الدهاريس


In [80]:
ff = cleaned_hadiths_from_graph[295]
for i in Hadiths_from_xml:
    if i in ff:
        print('here')

IndexError: list index out of range

In [81]:
c = 0
f = []
for index,i in enumerate(cleaned_hadiths_from_graph):
    for j in Hadiths_from_xml:
        if i == j:
            c+=1
            f.append(index)


In [82]:
print(set(f))
print(len(set(f)))
print(c)
298 - c

set()
0
0


298

In [ ]:
# a = str(s[3])
# print(a)
# A = removeDiacritics(a)
# B  = processText(A)
# regex pattern to get the text inside tags
pattern = "<[^>]*>(.*?)<\/[^>]*>"
count = 0
Nat = []
for i in s:
    T = processText(i)
    T = removeDiacritics(T)
#     T = removeSpace(T)
#     print(T)
    Nat.append(T)
    


### Graph Validation Functions
1. Chapters (suras)
2. Verses
3. Commentary Sections 
4. Hadiths
5. Not Hadiths
6. narrator
7. Topic/ SubTopic
8. Person,Location,Organization & Time
9. Verses
10. Poetry

# Helper Functions 

In [ ]:
# text processing function
def removeDiacritics(text):
    for a in pyarabic.araby.DIACRITICS:
        text = text.replace(a, '')
    return text


def processText(text):
    text = str(text)
    text = text.replace('\t', '')
    text = text.replace('\n', ' ')

    while text.find('  ') != -1:
        text = text.replace('  ', ' ')

    text = text.strip()

    return text


def removeSpace(text: str) -> str:
    while text.find(' ') != -1:
        text = text.replace(' ', '')

    return text

from collections import Counter

def lists_have_same_elements(list1, list2):
    counter1 = Counter(list1)
    counter2 = Counter(list2)
    
    return counter1 == counter2

## narrator validation

In [ ]:
def Validate_Narrator(graph_file,xml_file,nat_type):
    
    # Extracting Graph info (Narrators)
    
    # Create a new RDF graph
    graph = Graph()

    # Parse the RDF data from a file or URL
    graph.parse(graph_file)

    # Define the SPARQL query
    query = f"""
    PREFIX : <http://www.tafsirtabari.com/ontology#>
    PREFIX W3:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT DISTINCT *
    WHERE {{
      {{
      ?Person rdf:type :Narrator.
        ?Person :hasName ?name.
      ?Person :hasNarratorType :{nat_type}.
      }}
      union
      {{
          ?Person rdf:type :RootNarrator.
        ?Person :hasName ?name.
      ?Person :hasNarratorType :{nat_type}.
      }}
    }}
    """
    R = []
    # Execute the SPARQL query
    results = graph.query(query)
    # Process the query results
    for row in results:
        # Access the query variables
        person = row["Person"]
        name = row["name"]
#         print(f"Person: {person}, Name: {name}")
        R.append(str(name))

    narrators_from_graph = list(set(R))
    list_i = []

    for xml_f in xml_file:
        # extracting info from xml
        with open(xml_f, 'r', encoding='utf8') as f:     
            xml_file_as_string = f.read()

        Bs_data = BeautifulSoup(xml_file_as_string, "xml")

        tag = Bs_data.find_all("persName")
        root = ET.fromstring(xml_file_as_string)
        for i in tag:
        #     print(i)
            O = i.get("ana")
            if O == nat_type: # set narrator type
    #             print(i)
                list_i.append(i)

    s = set(list_i)
    S = list(s)
        
    # regex pattern to get the text inside tags
    pattern = "<[^>]*>(.*?)<\/[^>]*>"
    count = 0
    Nat = []
    for i in S:
        T = processText(i)
        T = removeDiacritics(T)
    #     T = removeSpace(T)
        output = re.findall(pattern, T)
    #     print(output)
        if len(output) != 0:
            count+=1
#             print(output[0])
            Nat.append(output[0])
        else:
            print("something wrong ",i)
    
    
    narrators_from_xml = list(set(Nat))

    if lists_have_same_elements(narrators_from_graph, narrators_from_xml):
        print("The lists have the same elements.")
        print("Narrators test passed! Validated! ✅ ")
    else:
        print("The lists do not have the same elements.")
    
    

In [69]:
a = ['D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\Dataset\\S025N\\sure_25_section_25.1-25.46.xml','D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\Dataset\\S025N\\sure_25_section_25.47-25.52.xml']
# g_file = "D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-9f4906f3913e4c1e2353d282ee31f7b4aafcab4b\\A-Box\\Tafsir Al-Tabari A-Box S100.owl"
g_file = 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-9f4906f3913e4c1e2353d282ee31f7b4aafcab4b\\A-Box\\Tafsir Al-Tabari A-Box S25.owl'

xml_file = a

Validate_Narrator(g_file,xml_file,"rawi")

The lists have the same elements.
Narrators test passed! Validated! ✅ 


## NER Validation

In [70]:
def Validate_NER(graph_file,xml_file,NER):
    
    # Extracting Graph info (Narrators)
    
    # Create a new RDF graph
    graph = Graph()

    # Parse the RDF data from a file or URL
    graph.parse(graph_file)
    
    NER_type = NER
    text = 'hasName'
    if NER == 'Time':
        text = 'hasTime'
    # Define the SPARQL query
    query = f"""
    PREFIX : <http://www.tafsirtabari.com/ontology#>
    PREFIX W3:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT DISTINCT ?individual ?Name
    WHERE {{
      
          ?individual rdf:type :{NER_type}.
            ?individual :{text} ?Name.
    }}
    
    """
    R = []
    # Execute the SPARQL query
    results = graph.query(query)
    # Process the query results
    for row in results:
        # Access the query variables
        org = row["individual"]
        name = row["Name"]
#         print(f"Organization: {org}, Name: {name}")
        R.append(str(name))

    NER_from_graph = list(set(R))

    # extracting info from xml
    for xml_f in xml_file:
        with open(xml_f, 'r', encoding='utf8') as f:     
            xml_file_as_string = f.read()

        Bs_data = BeautifulSoup(xml_file_as_string, "xml")

        tag = Bs_data.find_all("name")
        root = ET.fromstring(xml_file_as_string)
        list_i = []
        for i in tag:
        #     print(i)
            O = i.get("role")
            if O == NER_type.lower(): # set narrator type
    #             print(i)
                list_i.append(i.text)

    s = set(list_i)
    S = list(s)
#     print(len(S),S)
    print('\n\n')
    # regex pattern to get the text inside tags
    pattern = "<[^>]*>(.*?)<\/[^>]*>"
    count = 0
    Nat = []
    for i in S:
        
    #     T = removeSpace(T)
        T = processText(i)
        T = removeDiacritics(T)
        Nat.append(T)
    
    
    NER_from_xml = list(set(Nat))

    if lists_have_same_elements(NER_from_graph, NER_from_xml):
        print("The lists have the same elements.")
        print(f"{NER_type} test passed! Validated! ✅ ")
    else:
        print("The lists do not have the same elements.")
    
    

In [71]:
g_file = "D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-9f4906f3913e4c1e2353d282ee31f7b4aafcab4b\\A-Box\\Tafsir Al-Tabari A-Box S100.owl"
xml_file = 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Graph-Generator\\A-Box\\Dataset\\S100M\\sure_100_section_100.1-100.1.xml'
Validate_NER(g_file,xml_file,'Time')

FileNotFoundError: [Errno 2] No such file or directory: 'D'

## Poetry Validation

In [72]:
def Validate_Poetry(graph_file,xml_file):
    
    # Extracting Graph info (Narrators)
    
    # Create a new RDF graph
    graph = Graph()

    # Parse the RDF data from a file or URL
    graph.parse(graph_file)
    
    # Define the SPARQL query
    query = """
    PREFIX : <http://www.tafsirtabari.com/ontology#>
    PREFIX W3:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT DISTINCT *
    WHERE {

    ?s rdf:type :Poetry.
    ?s :hasText ?n.

    }
    """
    R = []
    # Execute the SPARQL query
    results = graph.query(query)
    # Process the query results
    for row in results:
        # Access the query variables
        s = row["s"]
        n = row["n"]
        print(f"s: {s}, n: {n}")
        R.append(str(n))

    poetry_from_graph = list(set(R))
        
    for xml_f in xml_file:
        # extracting info from xml
        with open(xml_f, 'r', encoding='utf8') as f:     
            xml_file_as_string = f.read()

        Bs_data = BeautifulSoup(xml_file_as_string, "xml")
        tag = Bs_data.find_all("quote")
        root = ET.fromstring(xml_file_as_string)
        for i in tag:
        #     print(i)
            O = i.get("type")
            if O == "poetry":
                print(processText(removeDiacritics(str(i))))
                print('\n\n')
                list_i.append(processText(removeDiacritics(i.text)))

    s = set(list_i)
    S = list(s)
    

    print('\n\n')
    # regex pattern to get the text inside tags
    pattern = "<[^>]*>(.*?)<\/[^>]*>"
    count = 0
    Nat = []
    for i in S:
        
    #     T = removeSpace(T)
        T = processText(i)
        T = removeDiacritics(T)
        Nat.append(T)
        
        
    # Further text cleaning (needed for pieces of text)
    #graph
    cleaned_poetry_from_graph = []
    for i in poetry_from_graph:
        a = remove_numbers_and_slashes_hadith(i)
        cleaned_poetry_from_graph.append(removeDiacritics(a))

    # xml
    list_i_p = []
    for i in list_i:
        filtered_list = remove_numbers_and_slashes(i)
        list_i_p.append(filtered_list)

    # removing leading spaces
    poetry_from_xml = remove_leading_spaces_list(list_i_p)

    cleaned_poetry_from_graph.sort()
    poetry_from_xml.sort()

    
    
    poetry_from_xml = list(set(Nat))

    if lists_have_same_elements(cleaned_poetry_from_graph, poetry_from_xml):
        print("The lists have the same elements.")
        print(f"Poetry test passed! Validated! ✅ ")
    else:
        print("The lists do not have the same elements.")
    
    

In [73]:
a = ['D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\Dataset\\S025N\\sure_25_section_25.1-25.46.xml','D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\Dataset\\S025N\\sure_25_section_25.47-25.52.xml']
g = 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\A-Box\\Tafsir Al-Tabari A-Box S25.owl'
Validate_Poetry(g,a)

s: http://www.tafsirtabari.com/ontology#P_001, n: إِذْ أُجَارِي الشيطان فِي سَنَنِ الْغَسيِّ وَمَنْ مَالَ مَيْلَهُ مَثْبُورُ
s: http://www.tafsirtabari.com/ontology#P_002, n: يَا رَسُولَ الْمَلِيكِ إِنَّ لِسَانِي رَاتِقٌ مَا فَتَّقْتُ إِذْ أَنَا بُورُ
s: http://www.tafsirtabari.com/ontology#P_003, n: حنت إلي النخلة القصوى فقلت لها حجر حرام ألا تلك الدهاريس
s: http://www.tafsirtabari.com/ontology#P_004, n: فهممت أن ألقي إليها محجرا فلمثلها يلقي إليه المحجر
s: http://www.tafsirtabari.com/ontology#P_005, n: وقدم الخوارج الضلال إلى عباد ربهم وقالوا إن دماءكم لنا حلال
s: http://www.tafsirtabari.com/ontology#P_006, n: سَبَقْتُ إِلَى فَرَطٍ ناهِلٍ تَنَابِلَةً يَحْفُرُونَ الرِّسَاسَا
s: http://www.tafsirtabari.com/ontology#P_007, n: حتى يقول الناس مما رأوا يا عجبا للميت الناشر
s: http://www.tafsirtabari.com/ontology#P_008, n: رعى بها مرج ربيع ممرجا
s: http://www.tafsirtabari.com/ontology#P_009, n: ألم يحزنك أن حبال قيس وتغلب قد تباينتا انقطاعا
s: http://www.tafsirtabari.com/ontology#P_010, n: 

NameError: name 'remove_numbers_and_slashes_hadith' is not defined

In [91]:
list_i = []
import re
# D:\University\semester6\KRR_WORK\FYP-AUTOKG-Narrator_fixed\Dataset\S114٘M
# a = ['D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\Dataset\\S025N\\sure_25_section_25.1-25.46.xml','D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\Dataset\\S025N\\sure_25_section_25.47-25.52.xml']
a = ['D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\Dataset\\S114٘M\\sure_114_section_114.1-114.1.xml']
verses = []

for xml_f in a:
    # extracting info from xml
    with open(xml_f, 'r', encoding='utf8') as f:     
        xml_file_as_string = f.read()
        Bs_data = BeautifulSoup(xml_file_as_string, 'xml')

    tags_with_quotes = Bs_data.find_all("head")
    
    for tag in tags_with_quotes:
        quote_tag = tag.find("quote")
        if quote_tag:
            l_tags = quote_tag.find_all("l")  # Find nested <l> tags
            if l_tags:
                for l_tag in l_tags:
                    l_text = l_tag.get_text()
                    verses.append(l_text)    
            else:        
                quote_text = quote_tag.get_text()  # Get the text within the quote tag
                verses.append(quote_text)
print("Total verses:", len(verses))

            
print(verses)      


Total verses: 6
['قُلْ أَعُوذُ بِرَبِّ النَّاسِ ', ' مَلِكِ النَّاسِ ', ' إِلَهِ النَّاسِ ', ' مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ ', ' الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ ', ' مِنَ الْجِنَّةِ وَالنَّاسِ ']


In [92]:
len(verses)

6

In [93]:
# querying from graph
    
# Create a new RDF graph
graph = Graph()

graph_file = 'D:\\University\\semester6\\KRR_WORK\\FYP-AUTOKG-Narrator_fixed\\A-Box\\Tafsir Al-Tabari A-Box S114.owl'


# Parse the RDF data from a file or URL
graph.parse(graph_file)

# Define the SPARQL query
query = """
PREFIX : <http://www.tafsirtabari.com/ontology#>
PREFIX W3:<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT *
WHERE {

 ?k rdf:type :Chapter.
?k :containsVerse ?v.
?v :hasText ?t.
}
"""
R = []
# Execute the SPARQL query
results = graph.query(query)
# Process the query results
for row in results:
    # Access the query variables
    s = row["k"]
    n = row["v"]
    t = row["t"]
    
    print(f"s: {s}, n: {n}, t: {t}")
    R.append(str(t))

hadiths_from_graph = list(set(R))

s: http://www.tafsirtabari.com/ontology#C_114, n: http://www.tafsirtabari.com/ontology#V114:002, t: مَلِكِ النَّاسِ
s: http://www.tafsirtabari.com/ontology#C_114, n: http://www.tafsirtabari.com/ontology#V114:003, t: إِلَٰهِ النَّاسِ
s: http://www.tafsirtabari.com/ontology#C_114, n: http://www.tafsirtabari.com/ontology#V114:004, t: مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
s: http://www.tafsirtabari.com/ontology#C_114, n: http://www.tafsirtabari.com/ontology#V114:005, t: الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ
s: http://www.tafsirtabari.com/ontology#C_114, n: http://www.tafsirtabari.com/ontology#V114:006, t: مِنَ الْجِنَّةِ وَالنَّاسِ
s: http://www.tafsirtabari.com/ontology#C_114, n: http://www.tafsirtabari.com/ontology#V114:001, t: بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ قُلْ أَعُوذُ بِرَبِّ النَّاسِ
